In [2]:
gf_exp = [0] * 512
gf_log = [0] * 256

def init_tables(prim=0x11d):
    '''Precompute the logarithm and anti-log tables for faster computation later, using the provided primitive polynomial.'''
    # prim is the primitive (binary) polynomial. Since it's a polynomial in the binary sense,
    # it's only in fact a single galois field value between 0 and 255, and not a list of gf values.
    global gf_exp, gf_log
    gf_exp = [0] * 512 # anti-log (exponential) table
    gf_log = [0] * 256 # log table
    # For each possible value in the galois field 2^8, we will pre-compute the logarithm and anti-logarithm (exponential) of this value
    x = 1
    for i in range(0, 255):
        gf_exp[i] = x # compute anti-log for this value and store it in a table
        gf_log[x] = i # compute log at the same time
        x <<= 1

        if (x & (1 << 8)):
            x ^= prim
    # Optimization: double the size of the anti-log table so that we don't need to mod 255 to
    # stay inside the bounds (because we will mainly use this table for the multiplication of two GF numbers, no more).
    for i in range(255, 509):
        gf_exp[i] = gf_exp[i - 255]
    return [gf_log, gf_exp]

In [3]:
tab = init_tables()
"""
Example 1:
_234_ = 2^22 (mod 285), _212_ = 2^41 (mod 285)
41 + 22 = 63
2^63 = 161
"""
print(f"234*212 = 2^(log(234) + log(212)) = 2^(041 + 022) = {tab[1][tab[0][234]+tab[0][212]]:03}")

"""
Example 2:
_251_ = 2^234 (mod 285), _121_ = 2^212 (mod 285)
234 + 212 = 446 === 191 (mod 285)
2^191 = 65
"""
print(f"251*121 = 2^(log(251) + log(121)) = 2^(234 + 212) = {tab[1][tab[0][251]+tab[0][121]]:03}")

"""
Example 3:
_142_ = 2^254 (mod 285)
254 + 254 = 508 === 253 (mod 285)
2^253 = 71
"""
print(f"142*142 = 2^(log(142) + log(142)) = 2^(254 + 254) = {tab[1][tab[0][142]+tab[0][142]]:03}")

234*212 = 2^(log(234) + log(212)) = 2^(041 + 022) = 161
251*121 = 2^(log(251) + log(121)) = 2^(234 + 212) = 065
142*142 = 2^(log(142) + log(142)) = 2^(254 + 254) = 071


In [4]:
def gf_mul(x,y):
    if x==0 or y==0:
        return 0
    return gf_exp[gf_log[x] + gf_log[y]] # should be gf_exp[(gf_log[x]+gf_log[y])%255] if gf_exp wasn't oversized

def gf_div(x,y):
    if y==0:
        print(f"Slow down that's not allowed. We're in Number Theory not Calculus for God's sake.")
    if x==0:
        return 0
    return gf_exp[(gf_log[x] + 255 - gf_log[y]) % 255] # Adding 255 is not needed in python, but it is in C.

In [5]:
"""
Modulo in python experimentation
"""
print(f"(234 + 167) % 255 = {(234 + 167) % 255:03}")
print(f"(234 + 167 - 255) % 255 = {(234 + 167) % 255:03}")
print(f"(234 + 167 - 255*12) % 255 = {(234 + 167) % 255:03}")
print(f"Python handles the modulo operator like a sane language")

(234 + 167) % 255 = 146
(234 + 167 - 255) % 255 = 146
(234 + 167 - 255*12) % 255 = 146
Python handles the modulo operator like a sane language


In [20]:
#print(f"{gf_mul(0x40, 0x12):x}")
print(f"{0x34 ^ 0xee:x}")
print(f"{0x56 ^ 0x2b:x}")

da
7d
